# Michelle's Data Capstone Project - Week 3

Peer Graded Review - Part 2

First section is just duplicating the work done in part 1 in separate notebook

In [17]:
import numpy as np 
import pandas as pd 
import requests 
from bs4 import BeautifulSoup
import re

sourcepage = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(sourcepage, "lxml")

right_table=soup.find('table', class_='wikitable sortable')

P=[]
B=[]
N=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        P.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        N.append(cells[2].find(text=True))
        
df_codes = pd.DataFrame(data=[P, B, N]).transpose()
df_codes.columns = ['Postal Code', 'Borough', 'Neighborhood']

df_codes.drop(df_codes[df_codes['Borough']=="Not assigned"].index, axis=0, inplace=True)
df1 = df_codes.reset_index()

df_group=df1.groupby('Postal Code').agg(lambda x: ','.join(x))
df_group.loc[df_group['Neighborhood']=="Not assigned",'Neighborhood']=df_group.loc[df_group['Neighborhood']=="Not assigned",'Borough']

df2 = df_group.reset_index()
df2['Borough']= df2['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")
df2.head(10)

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood\n,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae\n
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park\n,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West\n"
9,M1N,Scarborough,"Birch Cliff,Cliffside West\n"


After issues using geocoding, decided to import via the provided CSV file

In [18]:
df3 = pd.read_csv(r'https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv')
df3.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
df3.set_index("Postal Code")
df2.set_index("Postal Code")
postal_data=pd.merge(df2, df3)
postal_data.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood\n,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae\n,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park\n,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West\n",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West\n",43.692657,-79.264848
